In [ ]:
with open("toy_data.txt",'r',encoding='utf-8')as fn:
    lines = fn.readlines()
#     train = lines[:200000]
#     dev = lines[200000:400000]
    

In [ ]:
train_x = lines[1::2]
train_y = lines[0::2]
train_x = list(map(lambda x:x.replace("\n","").strip(),train_x))
train_y = list(map(lambda x:x.replace("\n","").strip(),train_y))

# dev_x = dev[1::2]
# dev_y = dev[0::2]
# dev_x = list(map(lambda x:x.replace("\n","").strip(),dev_x))
# dev_y = list(map(lambda x:x.replace("\n","").strip(),dev_y))

創建x和y的vocab

In [ ]:
vocab_x = set()
vocab_y = set()
train_x = list(map(lambda x:list(x),train_x))
train_y = list(map(lambda x:list(x),train_y))
# dev_x = list(map(lambda x:list(x),dev_x))
# dev_y = list(map(lambda x:list(x),dev_y))

In [ ]:
for s in train_x:
    vocab_x.update(s)
for s in train_y:
    vocab_y.update(s)

In [ ]:

with open('vocab_x','w',encoding='utf-8')as fno:
    for w in vocab_x:
        fno.write(w+"\n")
with open('vocab_y','w',encoding='utf-8')as fno:
    for w in vocab_y:
        fno.write(w+"\n")


In [1]:
w2idx_x = dict()
w2idx_y = dict()
with open('vocab_x','r',encoding='utf-8')as fn:
    lines = fn.readlines()
    for idx,w in enumerate(lines):
        w2idx_x[w[:-1]] = idx+2
with open('vocab_y','r',encoding='utf-8')as fn:
    lines = fn.readlines()
    for idx,w in enumerate(lines):
        w2idx_y[w[:-1]] = idx+2
idx2w_x = dict()
idx2w_y = dict()
for w, idx in w2idx_x.items():
    idx2w_x[idx] = w
for w, idx in w2idx_y.items():
    idx2w_y[idx] = w
idx2w_x[0] = "PAD"
idx2w_x[1] = "EOS"
idx2w_y[0] = "PAD"
idx2w_y[1] = "EOS"

In [ ]:
train_x_temp = list(map(lambda x:list(map(lambda y:w2idx_x[y],x)),train_x))
train_y_temp = list(map(lambda x:list(map(lambda y:w2idx_y[y],x)),train_y))
with open('train_x_vec','w',encoding='utf-8')as fno:
    for s in train_x_temp:
        fno.write(str(s)+'\n')
with open('train_y_vec','w',encoding='utf-8')as fno:
    for s in train_y_temp:
        fno.write(str(s)+'\n')

In [2]:
import ast
import numpy as np
from scipy.ndimage.interpolation import shift
from keras.preprocessing import sequence


Using TensorFlow backend.


In [3]:
def read_data(data_file):
    stories = []
    with open(data_file,'r',encoding='utf-8') as f:
        for line in f:
            line = ast.literal_eval(line.strip())
            stories.append(line)
    return stories

In [4]:
PAD = 0
EOS = 1

In [5]:
train_x = read_data('train_x_vec')
train_y = read_data('train_y_vec')
target_y = list(map(lambda x: x+[EOS],train_y))
train_y = list(map(lambda x: [EOS]+x,train_y))

In [6]:
train_x = sequence.pad_sequences(train_x, maxlen=100, dtype='int32',padding='post', truncating='post', value=0.)
train_y = sequence.pad_sequences(train_y, maxlen=100, dtype='int32',padding='post', truncating='post', value=0.)
target_y = sequence.pad_sequences(target_y, maxlen=100, dtype='int32',padding='post', truncating='post', value=0.)

### 開始建構graph和training

In [7]:
batch_size = 64
content_length=100
encoding_dim = 512
vocab_x_size = 2882+2
vocab_y_size = 1416+2
embedding_size = 100
readout_size = embedding_size

In [8]:
def get_next_batch():
    cur = 0
    pre = 0
    while(True):
        cur = cur+batch_size
        yield train_x[pre:cur],train_y[pre:cur],target_y[pre:cur]
        pre = cur
        cur = cur%448
        pre = pre%448

In [9]:
data_gen = get_next_batch()

In [10]:
import tensorflow as tf
X = tf.placeholder(tf.int32, [batch_size, content_length])

X_embeddings = tf.Variable(tf.random_uniform([vocab_x_size, embedding_size], -1.0, 1.0), dtype=tf.float32)
Y_embeddings = tf.Variable(tf.random_uniform([vocab_y_size, embedding_size], -1.0, 1.0), dtype=tf.float32)
X_inputs_embedded = tf.nn.embedding_lookup(X_embeddings, X)

In [11]:
W_s = tf.Variable(tf.random_uniform([2*encoding_dim, 2*encoding_dim], -1.0, 1.0), dtype=tf.float32)
W_b = tf.Variable(tf.random_uniform([2*encoding_dim,1,1], -1.0, 1.0), dtype=tf.float32)
U_s = tf.Variable(tf.random_uniform([2*encoding_dim, 2*encoding_dim], -1.0, 1.0), dtype=tf.float32)

def encoder():
    with tf.variable_scope('encode') as scope:
        X_lens = tf.reduce_sum(tf.sign(tf.abs(X)), 1)
        gru_cell_fw = tf.contrib.rnn.GRUCell(encoding_dim)
        gru_cell_bw = tf.contrib.rnn.GRUCell(encoding_dim)
        outputs, output_states = tf.nn.bidirectional_dynamic_rnn(
                                        gru_cell_fw,
                                        gru_cell_bw,
                                        X_inputs_embedded,
                                        sequence_length=X_lens,
                                        dtype=tf.float32)
        h = tf.concat(outputs,2)#[batch,time,2*encoding_dim]
        h = tf.reshape(h,[batch_size*content_length,2*encoding_dim])#[batch*time,2*encoding_dim]
        s = tf.concat(output_states,1)#[?,2*encoding_dim]
        WH = tf.matmul(W_s,tf.transpose(h))#[2*encoding_dim,batch*time]
        
        WH = tf.reshape(WH,[2*encoding_dim,batch_size,content_length])#[2*encoding_dim,batch,time]
        US = tf.matmul(U_s,tf.transpose(s))#[2*encoding_dim,?]
        US = tf.expand_dims(US,2)#[2*encoding_dim,?,1]
        
        sGate = tf.sigmoid(WH+US+W_b)#[2*encoding_dim,batch,time]
        
        h = tf.reshape(h,[batch_size,content_length,2*encoding_dim])#[batch,time,2*encoding_dim]

        sGate = tf.transpose(sGate,[1,2,0])#[batch,time,2*encoding_dim]
        h_hat = h*sGate#[batch,time,2*encoding_dim]
        
        return h_hat,output_states

In [12]:
W_d = tf.Variable(tf.random_uniform([encoding_dim, encoding_dim], -1.0, 1.0), dtype=tf.float32)
S_b = tf.Variable(tf.random_uniform([encoding_dim, 1], -1.0, 1.0), dtype=tf.float32)
W_a = tf.Variable(tf.random_uniform([encoding_dim, encoding_dim], -1.0, 1.0), dtype=tf.float32)
U_a = tf.Variable(tf.random_uniform([encoding_dim, 2*encoding_dim], -1.0, 1.0), dtype=tf.float32)
V_a = tf.Variable(tf.random_uniform([encoding_dim, 1], -1.0, 1.0), dtype=tf.float32)
W_r = tf.Variable(tf.random_uniform([2*readout_size, embedding_size], -1.0, 1.0), dtype=tf.float32)
U_r = tf.Variable(tf.random_uniform([2*readout_size, 2*encoding_dim], -1.0, 1.0), dtype=tf.float32)
V_r = tf.Variable(tf.random_uniform([2*readout_size, encoding_dim], -1.0, 1.0), dtype=tf.float32)
W_o = tf.Variable(tf.random_uniform([readout_size, vocab_y_size], -1.0, 1.0), dtype=tf.float32)


In [13]:
def step(pre_w,pre_c,pre_s,h_hat):
    with tf.variable_scope('step') as scope:
        #pre_w = [batch,embedding_size]
        #pre_c = [batch,2*encoding_dim]
        #pre_s = [batch,encoding_dim]
        #The paper doesn't clearly describe how to merge pre_w and pre_c into one array passed to gru_cell, 
        #so that I use concatenation operator to combinate them.
        gru_input = tf.concat([pre_w,pre_c],1)#[batch,embediing_size+2*encoding_dim]
        gru_cell = tf.contrib.rnn.GRUCell(encoding_dim)
        _,s_t = gru_cell(gru_input,pre_s)#[batch,encoding_dim]
        
        c_t = attention(pre_s,h_hat)#[batch,2*encoding_dim]
#         r_t = tf.matmul(W_r,tf.transpose(pre_w))#[2*readout_size,batch]
#         +tf.matmul(U_r,tf.transpose(pre_c))#[2*readout_size,batch]
#         +tf.matmul(V_r,tf.transpose(pre_s))#[2*readout_size,batch]
        r_t = tf.add(tf.add(tf.matmul(W_r,tf.transpose(pre_w)),tf.matmul(U_r,tf.transpose(pre_c))),tf.matmul(V_r,tf.transpose(pre_s)))
        r_t = tf.reshape(r_t,[2,readout_size,batch_size])#[2,readout_size,batch]
        m_t = tf.transpose(tf.reduce_max(r_t,0))#[batch,readout_size]
        p_t = tf.matmul(m_t,W_o)#[batch,vocab_y_size]
        #argmax_y = tf.argmax(p_t,1)#[batch]
        #argmax_y = tf.nn.embedding_lookup(Y_embeddings,argmax_y)#[batch,embedding_size]
        return c_t,s_t,p_t

In [14]:
def attention(pre_state,h_hat):
    #h_hat=[batch,time,2*encoding_dim]
    h_hat = tf.reshape(h_hat,[batch_size*content_length,2*encoding_dim])#[batch*time,2*encoding_dim]
    h_hat = tf.transpose(h_hat)#[2*encoding_dim,batch*time]
    #pre_state = [batch,encoding_dim]
    WS = tf.expand_dims(tf.matmul(W_a,tf.transpose(pre_state)),2)#[embedding_size,batch,1]

    UH = tf.reshape(tf.matmul(U_a,h_hat),[encoding_dim,batch_size,content_length])#[encoding_dim,batch,time]

    e_ti = tf.tanh(WS+UH)#[encoding_dim,batch,time]
  
    e_ti = tf.reshape(tf.transpose(e_ti,[1,2,0]),[batch_size*content_length,encoding_dim])#[batch*time,encoding_dim]
    e_ti = tf.matmul(e_ti,V_a)#[batch*time,1]
    e_ti = tf.reshape(e_ti,[batch_size,content_length])#[batch,time]
    a_ti = tf.nn.softmax(e_ti,1)#[batch,time]
    a_ti = tf.expand_dims(a_ti,2)#[batch,time,1]
    h_hat = tf.transpose(tf.reshape(h_hat,[2*encoding_dim,batch_size,content_length]),[1,0,2])#[batch,2*encoding_dim,time]
    
    c_t = tf.matmul(h_hat,a_ti)#[batch,2*encoding_dim,1]
    
    c_t = tf.squeeze(c_t)#[batch,2*encoding_dim]
    return c_t

In [15]:
Y = tf.placeholder(tf.int32, [batch_size, content_length])
trigger_Y = tf.placeholder(tf.int32, [batch_size,1])#[batch,1]

Y_inputs_embedded = tf.nn.embedding_lookup(Y_embeddings, Y)#[batch,time,embedding_size]
Y_trigger_inputs_embedding = tf.nn.embedding_lookup(Y_embeddings, trigger_Y)#[batch,1,embedding_size]
#target = tf.placeholder(tf.int32,[batch_size,content_length,vocab_y_size])

In [16]:
def decoder_training(h_hat,output_states):
    with tf.variable_scope('decode') as scope:
        pre_s = tf.tanh(tf.matmul(W_d,tf.transpose(output_states[1]))+S_b)#[encoding_dim,batch]
        pre_s = tf.transpose(pre_s)#[batch,encoding_dim]
        pre_c = attention(pre_s,h_hat)#[batch,2*encoding_dim]
        time_major_Y = tf.unstack(tf.transpose(Y_inputs_embedded,[1,0,2]),axis=0)#[time,batch,embedding_size]
        time_major_predict = []#[time,batch,vocab_y_size]
        for step_,pre_w in enumerate(time_major_Y):
            if step_ > 0:
                scope.reuse_variables()
            c_t,s_t,p_t = step(pre_w,pre_c,pre_s,h_hat)
            time_major_predict.append(p_t)
            pre_c = c_t
            pre_s = s_t
            #------------------when tesing, it must feed previous prediction Y into gru.
            #argmax_y = tf.argmax(p_t,1)#[batch]
            #argmax_y = tf.nn.embedding_lookup(Y_embeddings,argmax_y)#[batch,embedding_size]
        return time_major_predict#[time,batch,vocab_y_size]

In [17]:
def decoder_prediction(h_hat,output_states):
    with tf.variable_scope('decode') as scope:
        pre_s = tf.tanh(tf.matmul(W_d,tf.transpose(output_states[1]))+S_b)#[encoding_dim,batch]
        pre_s = tf.transpose(pre_s)#[batch,encoding_dim]
        pre_c = attention(pre_s,h_hat)#[batch,2*encoding_dim]
        time_major_Y = tf.unstack(tf.transpose(Y_trigger_inputs_embedding,[1,0,2]),axis=0)#[1,batch,embedding_size]
        time_major_predict = []#[time,batch,vocab_y_size]
        pre_w = tf.squeeze(time_major_Y)
        for step_ in range(content_length):
            if step_ > 0:
                scope.reuse_variables()
            c_t,s_t,p_t = step(pre_w,pre_c,pre_s,h_hat)
            time_major_predict.append(p_t)
            pre_c = c_t
            pre_s = s_t
            #------------------when tesing, it must feed previous prediction Y into gru.
            argmax_y = tf.argmax(p_t,1)#[batch]
            next_embedding_y = tf.nn.embedding_lookup(Y_embeddings,argmax_y)#[batch,embedding_size]
            pre_w = next_embedding_y
        return time_major_predict#[time,batch,vocab_y_size]

In [18]:
def get_trigger_y_batch(batch_size_):
    batch = np.zeros([batch_size_,1])#[batch,1]
    batch.fill(EOS)#[batch]
    return batch

In [19]:
def train():  
    h_hat,output_states = encoder()
    time_major_predict = decoder_training(h_hat,output_states)
    target = tf.placeholder(tf.int32,[batch_size,content_length])
    target_one_hot = tf.one_hot(target,vocab_y_size,1,0)#[batch_size,content_length,vocab_y_size]
    target_one_hot = tf.transpose(target_one_hot,[1,0,2])#[content_length,batch_size,vocab_y_size]
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_one_hot,logits=time_major_predict))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    grads_and_vars = optimizer.compute_gradients(loss)
    capped_grads_and_vars = [(tf.clip_by_value(g, -5.,5.), v) for g,v in grads_and_vars]
    train_op = optimizer.apply_gradients(capped_grads_and_vars)
    
    saver = tf.train.Saver()
 
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
 
        # writer = tf.summary.FileWriter()
        # 恢复前一次训练
        ckpt = tf.train.get_checkpoint_state('.')
        if ckpt != None:
            print(ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
        else:
            print("没找到模型")
 
        for step_ in range(20000):
            batch_train_x,batch_train_y,batch_target_y = data_gen.__next__()
                
            sess.run(train_op, feed_dict={X:batch_train_x, Y:batch_train_y, target:batch_target_y})
            losses = sess.run(loss,feed_dict={X:batch_train_x, Y:batch_train_y, target:batch_target_y})
            print(losses)
 
            # 保存模型并计算准确率
            if step_ % 1000 == 0:
                path = saver.save(sess, './machine_reading.model', global_step=step_)
                print(path)
 
                


In [20]:
def test():  
    h_hat,output_states = encoder()
    time_major_predict = decoder_prediction(h_hat,output_states)
    target = tf.placeholder(tf.int32,[batch_size,content_length])
    target_one_hot = tf.one_hot(target,vocab_y_size,1,0)#[batch_size,content_length,vocab_y_size]
    target_one_hot = tf.transpose(target_one_hot,[1,0,2])#[content_length,batch_size,vocab_y_size]
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_one_hot,logits=time_major_predict))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    grads_and_vars = optimizer.compute_gradients(loss)
    capped_grads_and_vars = [(tf.clip_by_value(g, -5.,5.), v) for g,v in grads_and_vars]
    train_op = optimizer.apply_gradients(capped_grads_and_vars)
    
    saver = tf.train.Saver()
 
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
 
        # writer = tf.summary.FileWriter()
        # 恢复前一次训练
        ckpt = tf.train.get_checkpoint_state('.')
        if ckpt != None:
            print(ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
        else:
            print("没找到模型")
 
        
        batch_train_x,batch_train_y,batch_target_y = data_gen.__next__()
        trigger_y_batch = get_trigger_y_batch(batch_size)#[batch,1]
        
        time_major_predict = sess.run(time_major_predict,
                 feed_dict={X:batch_train_x, 
                            Y:batch_train_y, 
                            target:batch_target_y,
                            trigger_Y:trigger_y_batch})#[time,batch,vocab_y_size]
        
        time_major_predict = np.stack(time_major_predict)#[time,batch,vocab_y_size]
        time_major_predict = tf.argmax(time_major_predict,2)#[time,batch]
        print(time_major_predict.shape)
        time_major_predict = sess.run(time_major_predict)#tensor to ndarray
        batch_major_predict = np.transpose(time_major_predict,[1,0])#[batch,time]
        print(batch_major_predict.shape)
        #batch_major_predict = np.squeeze(batch_major_predict)#[batch,time]
        for row in np.nditer(batch_major_predict,flags=['external_loop'],order='F'):
            #row = np.squeeze(row)#[time]
            words = list(map(lambda x:idx2w_y[x],row))
            print(''.join(words))
        #losses = sess.run(loss,feed_dict={X:batch_train_x, Y:batch_train_y, target:batch_target_y})
        #print(losses)
 
            
 

In [21]:
test()

.\machine_reading.model-1000
INFO:tensorflow:Restoring parameters from .\machine_reading.model-1000
(100, 64)
(64, 100)
日手G旗大持英英全調華亞相亞觀緊傳樂日三中宏英深F分三思加南美陸巴美鴻諾A停樂英福觀福英i蘋物C雲企D達T蘋抓指騰非陸美C創電3本機o下陸續政特球研為太中馬察跟南金亞星國達特港e析星科州非國電西國海基T損金特特察特特P果聯o端業a美o果住紋訊洲1國h業動D再遊o易通拓府爾創：在電線遜：台韓2化電郵電爾通d：再傳再礦路動總副、亞＆賣電爾：：、爾h取網g服財r航y、大辨與行0大e如腳列興戲g華富展批：新S美信上收英積IQY子政推發實官英發近度工易車統總騰證T單子展2V百、o代商n務務k空o三陸識C動月型i馬踏印策吸l電微電准將指o國大遊購特電l'A副宣展表施員特邀期爆齊斯廠D統訊實班出手示0e度AnI機i新管t全t星民新M服起銀l拉車新略引e、電氣建讓數n首手戲遊爾步j1G會布新J方預爾請擬發聚安威iJ領將表籠機無2l1ReB廣t創理r球a9眾創C務允行 松風創2玩推長停解造新排y次筆商戲晶伐i6專長與硬o案告押函裁森魯那馬lo投於露　晶線1o.M庫M布i：妙a大發月對：聯需許採W　靡：0家出華牌決全的名手推自機社圓　n手利權滴體u獲最寶　員林斯州完me　4餡美片V年d5大存成　vO招c當展的日L姻求貨撙o解大F1　D科交方球U出機出建　群代K取機訴五滴頻l批快MG1大登史成a 印Q　元轉R針y億和呆為市er　e機人「本i　大車節r決陸o6騰u將易案最S爐A榮網群公工L得部訟鉉出繁e　9Re.火堡無A B即重i兌單頭對n美解料企場Sb小網　工腕商q新增在措l問　r以訊o在　　大B　p耀路創司結AT門　　行　晶大月盼a4EOS悼前輪Ri時返P日英盔叫e元　風業拓ci家路暴智」品u公　4施d題美mI第 台傳伊離 大p品　將C盟-e裁在今建今片陸便重r萬於念例融od通手h圓特P車為投獲險採展at庭安露慧力熱i司惟省　w與國lo2A上醞頓岸T陸當牌採量uATs減德年立年模實可返 人大馬的資ue訊機o貶爾r服何資授大購需le也全航　之愛d有商城用i成卻aT季p市釀看風y首機手購產rRel4國上戰下組現能榮S　火里洪　sn新、n破　o務受V權　數